# Acompanhamento - Atividades Jira

## GRUPO SEI

---

### Bibliotecas e informações iniciais

**Edite apenas aqui!**

In [10]:
from datetime import date, datetime, timedelta
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import requests
import plotly
import plotly.graph_objects as go

SPRINT = "02"
DATA_INICIAL = "30-01-2023"
DATA_FINAL = "17-02-2023"
FILTER_ID = 10377


# --------------------------------------------------
# Jira
# Gere seu token pessoal aqui: https://id.atlassian.com/manage-profile/security/api-tokens
# --------------------------------------------------
user='patricia.nallin@cnpem.br'
authToken = ''
# --------------------------------------------------

### Coleta de dados via Jira API

In [11]:
# DATAS
HOJE = date.today().strftime("%d-%m-%Y")   
startday = datetime.strptime(DATA_INICIAL, "%d-%m-%Y")
endday = datetime.strptime(DATA_FINAL, "%d-%m-%Y")
today = datetime.strptime(HOJE, "%d-%m-%Y")

# --------------------------------------------------
# COLETA DE DADOS VIA API
# --------------------------------------------------
url = f'https://cnpem.atlassian.net/sr/jira.issueviews:searchrequest-csv-all-fields/{FILTER_ID}/SearchRequest-{FILTER_ID}.csv?atl_token=50900f97-f628-45cb-a7f4-b46d9a88f7c9_0830deba9bb0b468e2fa6057a022c670d36e43c7_lin'
download = requests.get(url, auth=(f'{user}', f'{authToken}'))
csv = download.content.decode('utf-8')
# --------------------------------------------------



# --------------------------------------------------
# Salvar em arquivo
# Se ultimo dia do sprint: salva em arquivo final
# Se dia intermediario, salva/sobrescreve arquivo de acompanhamento
# --------------------------------------------------
if (today == endday) and (today <= endday):
    f = open("Sprint_{}_{}.csv".format(SPRINT,DATA_FINAL), "w")
else:
    f = open("Sprint_{}_Acompanhamento.csv".format(SPRINT), "w")
f.write(csv)
f.close()
# --------------------------------------------------

### Leitura do CSV

In [12]:
# --------------------------------------------------
# DATAS
# --------------------------------------------------
HOJE = date.today().strftime("%d-%m-%Y")   
startday = datetime.strptime(DATA_INICIAL, "%d-%m-%Y")
endday = datetime.strptime(DATA_FINAL, "%d-%m-%Y")
today = datetime.strptime(HOJE, "%d-%m-%Y")


# --------------------------------------------------
# Leitura do CSV
# --------------------------------------------------
if (today >= endday):
    linhas = open("Sprint_{}_{}.csv".format(SPRINT,DATA_FINAL),'r').read()
else:
    linhas = open("Sprint_{}__Acompanhamento.csv".format(SPRINT),'r').read()

fields = (linhas.split("Status Category\n")[0]+"Status Category").split(',')
linhas = linhas.split("Status Category\n")[1]
linhas = linhas.replace("Done\n","Done\n\r\r").replace("In Progress\n","In Progress\n\r\r").replace("To Do\n","To Do\n\r\r")
linhas = linhas.replace(", "," ")
atividade = linhas.split("\n\r\r")[:-1]
# --------------------------------------------------

### Tratamento dos dados e informações a coletar

In [13]:
# --------------------------------------------------
# Variáveis a analisar
# --------------------------------------------------
projetos = []
totalPontos = 0
membros = {
            "Bruno":{"pontos":0, "natividades":0, "id":"63d01fd6c565900ff40519b9"},
            "Eduarda":{"pontos":0, "natividades":0, "id":"62bb04adde439ce00ee39aa9"},
            "Gabriel":{"pontos":0, "natividades":0, "id":"70121:016a6576-395b-4462-b1e7-fb210bae9d77"},
            "Marcelo":{"pontos":0, "natividades":0, "id":"62bb04b09e6ba34c9935f3f1"},
            "Patricia":{"pontos":0, "natividades":0, "id":"62b0acb8f38b4dcf73d839ec"},
            "Ramon":{"pontos":0, "natividades":0, "id":"62e90951e50f2f2a39548aa1"},
            "Robert":{"pontos":0, "natividades":0, "id":"62bf029efbc1f7c647b9f327"},
            "Rogério":{"pontos":0, "natividades":0, "id":"62bb024e018dea0d4073dbdd"},
            "Samuel":{"pontos":0, "natividades":0, "id":"62b0928ba58208122d9fbcb5"},
          }

status = {
            "Done":{"pontos":0, "natividades":0},
            "In Progress":{"pontos":0, "natividades":0},
            "To Do":{"pontos":0, "natividades":0}
         }

scrum_days = {}
for i in range(15):
    scrum_days[startday + timedelta(days=(7*int(i/5)+(i%5)))] = {"nday":i+1,
                                                                  "points":0,
                                                                  "integrated points":0}

# --------------------------------------------------
# TRATAMENTO DOS DADOS - POR ATIVIDADE (Cada atividade é um item da lista)
# --------------------------------------------------

for i in range(len(atividade)):
    proj = atividade[i].split(",")[fields.index("Project key")]
    totalPontos += float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
    if proj not in projetos:
        projetos.append(proj)
        
    for pessoa in membros.keys():
        if (pessoa in atividade[i].split(",")[fields.index("Assignee")]) or (membros[pessoa]["id"] in atividade[i].split(",")[fields.index("Description")]):
            membros[pessoa]['pontos'] += float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
            membros[pessoa]['natividades'] += 1
            
    for stat in status.keys():
        if (stat in atividade[i].split(",")[fields.index("Status Category")]):
            status[stat]['pontos'] += float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
            status[stat]['natividades'] += 1
        
     
    if atividade[i].split(",")[fields.index("Resolved")] != "":
        resolved = datetime.strptime(atividade[i].split(",")[fields.index("Resolved")].split(" ")[0], "%d/%b/%y")
        try:
            scrum_days[resolved]["points"] += float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
        except:
            continue
# --------------------------------------------------
    
    
# --------------------------------------------------
# PONTOS INTEGRADOS - BURNUP
# --------------------------------------------------
for day in scrum_days.keys():
    if (day == startday):
        scrum_days[day]["integrated points"] = scrum_days[day]["points"]
    else:
        previous_day = list(scrum_days.keys())[scrum_days[day]["nday"]-2]
        scrum_days[day]["integrated points"] = scrum_days[day]["points"] + scrum_days[previous_day]["integrated points"]
# --------------------------------------------------

### Plots

In [14]:
# --------------------------------------------------
# PLOTS
# --------------------------------------------------    
plot_data = {"date":[], "burnup":[], "burndown":[], "daily":[]}
for day in scrum_days.keys():
    plot_data["date"].append(day)
    plot_data["burnup"].append(scrum_days[day]["integrated points"])
    plot_data["burndown"].append(int(totalPontos - scrum_days[day]["integrated points"]))
    plot_data["daily"].append(int(scrum_days[day]["points"]))

    
# --------------------------------------------------
# BURNDOWN
# --------------------------------------------------  
fig = go.Figure()
fig.add_trace(go.Scatter(x=[startday, endday], y=[totalPontos, 0],
                    mode='lines',
                    name='ideal'))
fig.add_trace(go.Scatter(x=plot_data["date"], y=plot_data["burndown"],
                    mode='lines+markers',
                    name='Pontos a concluir'))
fig.update_layout(
    title={
        'text': "Burndown Completo - Grupo SEI - Sprint {}/{}".format(SPRINT, HOJE[-4:]),
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis_title="Pontos restantes",
)
fig.show()
plotly.offline.plot(fig, filename = 'burndown_SEI_Sprint_{}_{}.html'.format(SPRINT, HOJE[-4:]), auto_open=False)



# --------------------------------------------------
# BURNUP
# --------------------------------------------------  
fig = go.Figure()
fig.add_trace(go.Scatter(x=[startday, endday], y=[0, totalPontos],
                    mode='lines',
                    name='ideal'))
fig.add_trace(go.Scatter(x=plot_data["date"], y=plot_data["burnup"],
                    mode='lines+markers',
                    name='Pontos já queimados'))
fig.update_layout(
    title={
        'text': "Burnup Completo - Grupo SEI - Sprint {}/{}".format(SPRINT, HOJE[-4:]),
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis_title="Pontos restantes",
)
fig.show()
plotly.offline.plot(fig, filename = 'burnup_SEI_Sprint_{}_{}.html'.format(SPRINT, HOJE[-4:]), auto_open=False)


'burnup_SEI_Sprint_02_2023.html'

### Gráficos de pizza/donuts

In [15]:
# --------------------------------------------------
# GRAFICO DONUTS - PERCENTUAL
# --------------------------------------------------  
labels = list(status.keys())
pts = []
atv = []
for stat in status.keys():
    pts.append(int(status[stat]["pontos"]))
    atv.append(int(status[stat]["natividades"]))

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=pts, name="Pontos"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=atv, name="Atividades"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")
fig.update_layout(
    title_text="Status Geral - Grupo SEI - Sprint {}/{}".format(SPRINT, HOJE[-4:]),
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Pontos', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Atividades', x=0.84, y=0.5, font_size=20, showarrow=False)])
fig.show()

plotly.offline.plot(fig, filename = 'pieChart_SEI_Sprint_{}_{}.html'.format(SPRINT, HOJE[-4:]), auto_open=False)


'pieChart_SEI_Sprint_02_2023.html'

### Números

In [16]:
print("-----  SPRINT {}/{}         ({})  -----".format(SPRINT, HOJE[-4:], HOJE))            
print("\nTOTAL DE PROJETOS ENT: {}\n\n".format(len(projetos)))



print("\nSTATUS\t\t#PONTOS\t\t% PONTOS\t#ATIVIDADES\t% ATIVIDADES")
for stat in status.keys():
    print("{}\t{}\t\t{:.1f}\t\t{}\t\t{:.1f}".format(stat.ljust(8, " "), int(status[stat]["pontos"]), 100*status[stat]["pontos"]/totalPontos,int(status[stat]["natividades"]), 100*status[stat]["natividades"]/len(atividade)))
print("Total\t\t{}\t\t{}".format(int(totalPontos), len(atividade)))



print("\n\nMEMBRO\t\t#PONTOS\t\t#ATIVIDADES")
for pessoa in membros.keys():
    print("{}\t{}\t\t{}".format(pessoa.ljust(8, " "), int(membros[pessoa]["pontos"]), membros[pessoa]["natividades"]))


-----  SPRINT 02/2023         (17-02-2023)  -----

TOTAL DE PROJETOS ENT: 12



STATUS		#PONTOS		% PONTOS	#ATIVIDADES	% ATIVIDADES
Done    	332		79.8		28		75.7
In Progress	68		16.3		6		16.2
To Do   	16		3.8		3		8.1
Total		416		37


MEMBRO		#PONTOS		#ATIVIDADES
Bruno   	52		4
Eduarda 	24		3
Gabriel 	181		13
Marcelo 	100		8
Patricia	105		11
Ramon   	105		9
Robert  	120		11
Rogério 	142		9
Samuel  	60		4
